In [36]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
from scipy.optimize import curve_fit

pio.templates.default = "plotly_white"

$$
\ln\left(\cfrac{t_1-t_0}{t_0} \cdot \cfrac{1}{\sqrt{T}}\right) = \ln\left(\cfrac{\tau_0}{\varrho} \cdot \sqrt{\cfrac{2R}{\pi M}}\right) + \cfrac{Q}{R}\cdot\cfrac{1}{T}
$$

In [18]:
def y(t1, t0, T):
    return np.log((t1-t0)/t0 /np.sqrt(T))

df = pd.DataFrame([
    ['water', 210, 130, 'aer130-2'],
    ['water', 208, 130, 'alc130'],
    ['water', 164, 140, 'alc140'],
    ['meto', 220, 140, 'alc140'],
    ['water', 131, 150, 'alc150'],
    ['meto', 168, 150, 'alc150'],
    ['etho', 320, 150, 'alc150'],
    ['water', 107, 160, 'alc160-2'],
    ['meto', 133, 160, 'alc160-2'],
    ['etho', 239, 160, 'alc160-2'],
    ['inert', 21, 130, 'aer130-2'],
    ['inert', 21, 130, 'alc130'],
    ['inert', 20, 140, 'alc140'],
    ['inert', 20, 150, 'alc150'],
    ['inert', 19, 160, 'alc160'],
], columns=['sample', 't', 'T', 'exp'])

df_alc = df[df['exp'].str.contains('alc')]

In [91]:
df_x = df_alc.pivot(index='T', columns='sample', values='t').reset_index()
x = 1/(df_x['T']+273.15)
t0 = df_x['inert']
t1water = df_x['water']
t1meto = df_x['meto']
t1etho = df_x['etho']
ywater = y(t1water, t0, df_x['T']+273.15)
ymeto = y(t1meto, t0, df_x['T']+273.15)
yetho = y(t1etho, t0, df_x['T']+273.15)

kwater, bwater = np.polyfit(x, ywater, 1)
kmeto, bmeto = np.polyfit(x[1:], ymeto[1:], 1)
ketho, betho = np.polyfit(x[2:], yetho[2:], 1)

xs = np.linspace(x.min(), x.max(), 20)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=ywater, mode='markers', name='water', legendgroup='water'))
fig.add_trace(go.Scatter(x=x, y=ymeto, mode='markers', name='meto', legendgroup='meto'))
fig.add_trace(go.Scatter(x=x, y=yetho, mode='markers', name='etho', legendgroup='etho'))

fig.add_trace(go.Scatter(x=xs, y=kwater*xs+bwater, mode='lines', name=f'water: {kwater:.2f}x+{bwater:.2f}', legendgroup='water'))
fig.add_trace(go.Scatter(x=xs, y=kmeto*xs+bmeto, mode='lines', name=f'meto: {kmeto:.2f}x+{bmeto:.2f}', legendgroup='meto'))
# fig.add_trace(go.Scatter(x=xs, y=ketho*xs+betho, mode='lines', name=f'etho: {ketho:.2f}x+{betho:.2f}', legendgroup='etho'))

fig.update_layout(
    xaxis_title='1/T, 1/K',
    yaxis_title='ln((t1-t0)/t0/sqrt(T))',
    title='Проверка зависимости времени выхода от температуры',
    height=600,
    width=1200,
)
fig.write_image('temperature.png')
fig.show()

In [29]:
Qwater = kwater*8.314
Qmeto = kmeto*8.314
Qetho = ketho*8.314

print(f'Qwater = {Qwater/1e3:.2f} kJ/mol\nQmeto = {Qmeto/1e3:.2f} kJ/mol\nQetho = {Qetho/1e3:.2f} kJ/mol')

Qwater = 34.00 kJ/mol
Qmeto = 39.81 kJ/mol
Qetho = 41.23 kJ/mol


## Теперь данные которые не наши

In [84]:
T = 170 + 273.15  # K
df_speed = pd.DataFrame([
    ['water', 7 , 4*60+29, 31],
    ['meto' , 7 , 5*60+26, 28],
    ['etho' , 7 , 9*60+20, 44],
    
    ['water', 12, 2*60+58, 19],
    ['meto' , 12, 3*60+37, 22],
    ['etho' , 12, 6*60+14, 32],
    
    # ['water', 20, 2*60+ 2, 13],
    ['meto' , 20, 2*60+28, 14],
    ['etho' , 20, 4*60+16, 21],
    
    ['water', 30, 1*60+30,  8],  # Seems like very wrong
    # ['meto' , 30, 1*60+45, 11],
    ['etho' , 30, 3*60+ 4, 17],
    
    ['water', 40, 1*60+14,  7],
    ['meto' , 40, 1*60+30,  8],
    ['etho' , 40, 2*60+36, 13],
    
    ['water', 50, 1*60+ 3,  6],
    ['meto' , 50, 1*60+17,  7],
    ['etho' , 50, 2*60+14, 13],
    
    ['water', 60, 0*60+56,  6],
    ['meto' , 60, 1*60+ 8,  7],
    ['etho' , 60, 1*60+59, 14],
], columns=['sample', 'speed', 't', 'dt'])
df_speed['res'] = df_speed['t'] / df_speed['dt']
df_speed['NTP'] = 4*df_speed['res']**2
df_speed['H'] = 200/df_speed['NTP']

# df_speed_x = df_speed.pivot(index='speed', columns='sample', values='res').reset_index()

In [85]:
def func(x, a, b, c):
    return a/x + b*x + c

In [86]:
popts = {}
for sample in ('water', 'meto', 'etho'):
    df_speed_sample = df_speed[(df_speed['sample'] == sample) & (df_speed['speed'] > 10)]
    popt, pcov = curve_fit(func, df_speed_sample['speed'], df_speed_sample['H'])
    popts[sample] = popt
    print(f'{sample}: {popt}')

water: [ 6.84619218  0.00923867 -0.11101539]
meto: [4.86371029 0.00609765 0.04825214]
etho: [ 4.75556579  0.01176413 -0.15678162]


In [90]:
xs = np.linspace(8, 70, 100)

colors = px.colors.qualitative.Plotly[:3]

fig = go.Figure()
for i, sample in enumerate(['water', 'meto', 'etho']):
    fig.add_trace(go.Scatter(marker_color=colors[i], x=df_speed[df_speed['sample'] == sample]['speed'], y=df_speed[df_speed['sample'] == sample]['H'], mode='markers', name=sample+' data', legendgroup=sample))
    fig.add_trace(go.Scatter(line_color=colors[i], x=xs, y=func(xs, *popts[sample]), mode='lines', name=sample+' fit', legendgroup=sample))
    print(f"{sample} optimal speed: {(popts[sample][0]/popts[sample][1])**0.5:.0f} mm/hr")
fig.update_layout(
    xaxis_title='speed, mm/hr',
    yaxis_title='H, cm',
    title=f'speed - Theoretical Plate Height',
    width=1200,
    height=600,
)
fig.write_image('speed.png')
fig.show()


water optimal speed: 27 mm/hr
meto optimal speed: 28 mm/hr
etho optimal speed: 20 mm/hr
